In [52]:
# pip install pyodbc

In [53]:
import pandas as pd
import pyodbc
import os
from datetime import datetime

In [54]:
# Define the parameters for connect to SQL Server database
driver_name = "ODBC Driver 17 for SQL Server"
server_name = "TIT-20230930FXD\\SQLEXPRESS"
database_name = "brazilian_ecommerce" # using database_name = "master" for create new database
trusted_connection = "yes"

# Define connection string
conn_str = f"DRIVER={driver_name};SERVER={server_name};DATABASE={database_name};Trusted_Connection={trusted_connection}"

In [55]:
# Read CSV file
data_path = r"C:\Users\Admin\Documents\GitHub\data-science-bootcamp9\09_Essential Python for Data Analyst\Project - Automatically Create and add data to a SQL Server\data"
file_name = "olist_order_reviews_dataset.csv"
df = pd.read_csv(os.path.join(data_path, file_name), encoding="utf8")
df.info()

review_id                  object
order_id                   object
review_score                int64
review_comment_title       object
review_comment_message     object
review_creation_date       object
review_answer_timestamp    object
dtype: object

In [56]:
df.duplicated().sum()

0

In [57]:
df.drop_duplicates(inplace=True)

In [58]:
df.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


In [59]:
# Convert "review_answer_timestamp" column to datetime type
df["review_creation_date"] = pd.to_datetime(df["review_creation_date"])

In [60]:
# Convert "review_answer_timestamp" column to datetime type
df["review_answer_timestamp"] = pd.to_datetime(df["review_answer_timestamp"])

In [61]:
df = df.astype(object)
df.dtypes

review_id                  object
order_id                   object
review_score               object
review_comment_title       object
review_comment_message     object
review_creation_date       object
review_answer_timestamp    object
dtype: object

In [62]:
# Convert NaN values to None
df = df.where(pd.notnull(df), None)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99224 entries, 0 to 99223
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   review_id                99224 non-null  object
 1   order_id                 99224 non-null  object
 2   review_score             99224 non-null  object
 3   review_comment_title     11568 non-null  object
 4   review_comment_message   40977 non-null  object
 5   review_creation_date     99224 non-null  object
 6   review_answer_timestamp  99224 non-null  object
dtypes: object(7)
memory usage: 5.3+ MB


In [63]:
df.nunique()

review_id                  98410
order_id                   98673
review_score                   5
review_comment_title        4527
review_comment_message     36159
review_creation_date         636
review_answer_timestamp    98248
dtype: int64

In [64]:
print(df.review_id.astype(str).str.len().max())
print(df.order_id.astype(str).str.len().max())
print(df.review_comment_title.astype(str).str.len().max())
print(df.review_comment_message.astype(str).str.len().max())

32
32
26
208


In [65]:
# drop_table_query = """
#     DROP TABLE IF EXISTS order_reviews
# """

# create_table_query = """
#     CREATE TABLE order_reviews(    
#         review_id VARCHAR(50) NOT NULL,
#         order_id VARCHAR(50) NOT NULL,
#         review_score INT NULL,
#         review_comment_title NVARCHAR(255) NULL,
#         review_comment_message NVARCHAR(255) NULL,
#         review_creation_date DATETIME NULL,
#         review_answer_timestamp DATETIME NULL
#     );
# """

# with pyodbc.connect(conn_str) as con:
#     con.execute(drop_table_query)
#     con.execute(create_table_query)
#     con.commit()

In [66]:
# Define SQL command for truncate table
truncate_query = """
    TRUNCATE TABLE order_reviews
"""

# Execute command SQL for truncate table
with pyodbc.connect(conn_str) as con:
    cursor = con.cursor()
    cursor.execute(truncate_query)
    con.commit()

In [67]:
# Convert review_score column to string
df["review_score"] = df["review_score"].astype(str)

with pyodbc.connect(conn_str) as con:
    cursor = con.cursor()
    for index, row in df.iterrows():
        insert_query = """
            INSERT INTO order_reviews
            VALUES (?, ?, ?, ?, ?, ?, ?)
        """
        # Execute command SQL for insert new data
        cursor.execute(insert_query, tuple(row))
    con.commit()

In [68]:
with pyodbc.connect(conn_str) as con:
    df_query = pd.read_sql("SELECT * FROM order_reviews", con)
    
df_query.info()

C:\Users\Admin\AppData\Local\Temp\ipykernel_5400\3070648877.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_query = pd.read_sql("SELECT * FROM order_reviews", con)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99224 entries, 0 to 99223
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   review_id                99224 non-null  object        
 1   order_id                 99224 non-null  object        
 2   review_score             99224 non-null  int64         
 3   review_comment_title     11568 non-null  object        
 4   review_comment_message   40977 non-null  object        
 5   review_creation_date     99224 non-null  datetime64[ns]
 6   review_answer_timestamp  99224 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int64(1), object(4)
memory usage: 5.3+ MB
